# Machine Learning Trading Bot

In this Challenge, you’ll assume the role of a financial advisor at one of the top five financial advisory firms in the world. Your firm constantly competes with the other major firms to manage and automatically trade assets in a highly dynamic environment. In recent years, your firm has heavily profited by using computer algorithms that can buy and sell faster than human traders.

The speed of these transactions gave your firm a competitive advantage early on. But, people still need to specifically program these systems, which limits their ability to adapt to new data. You’re thus planning to improve the existing algorithmic trading systems and maintain the firm’s competitive advantage in the market. To do so, you’ll enhance the existing trading signals with machine learning algorithms that can adapt to new data.

## Instructions:

Use the starter code file to complete the steps that the instructions outline. The steps for this Challenge are divided into the following sections:

* Establish a Baseline Performance

* Tune the Baseline Trading Algorithm

* Evaluate a New Machine Learning Classifier

* Create an Evaluation Report

#### Establish a Baseline Performance

In this section, you’ll run the provided starter code to establish a baseline performance for the trading algorithm. To do so, complete the following steps.

Open the Jupyter notebook. Restart the kernel, run the provided cells that correspond with the first three steps, and then proceed to step four. 

1. Import the OHLCV dataset into a Pandas DataFrame.

2. Generate trading signals using short- and long-window SMA values. 

3. Split the data into training and testing datasets.

4. Use the `SVC` classifier model from SKLearn's support vector machine (SVM) learning method to fit the training data and make predictions based on the testing data. Review the predictions.

5. Review the classification report associated with the `SVC` model predictions. 

6. Create a predictions DataFrame that contains columns for “Predicted” values, “Actual Returns”, and “Strategy Returns”.

7. Create a cumulative return plot that shows the actual returns vs. the strategy returns. Save a PNG image of this plot. This will serve as a baseline against which to compare the effects of tuning the trading algorithm.

8. Write your conclusions about the performance of the baseline trading algorithm in the `README.md` file that’s associated with your GitHub repository. Support your findings by using the PNG image that you saved in the previous step.

#### Tune the Baseline Trading Algorithm

In this section, you’ll tune, or adjust, the model’s input features to find the parameters that result in the best trading outcomes. (You’ll choose the best by comparing the cumulative products of the strategy returns.) To do so, complete the following steps:

1. Tune the training algorithm by adjusting the size of the training dataset. To do so, slice your data into different periods. Rerun the notebook with the updated parameters, and record the results in your `README.md` file. Answer the following question: What impact resulted from increasing or decreasing the training window?

> **Hint** To adjust the size of the training dataset, you can use a different `DateOffset` value&mdash;for example, six months. Be aware that changing the size of the training dataset also affects the size of the testing dataset.

2. Tune the trading algorithm by adjusting the SMA input features. Adjust one or both of the windows for the algorithm. Rerun the notebook with the updated parameters, and record the results in your `README.md` file. Answer the following question: What impact resulted from increasing or decreasing either or both of the SMA windows?

3. Choose the set of parameters that best improved the trading algorithm returns. Save a PNG image of the cumulative product of the actual returns vs. the strategy returns, and document your conclusion in your `README.md` file.

#### Evaluate a New Machine Learning Classifier

In this section, you’ll use the original parameters that the starter code provided. But, you’ll apply them to the performance of a second machine learning model. To do so, complete the following steps:

1. Import a new classifier, such as `AdaBoost`, `DecisionTreeClassifier`, or `LogisticRegression`. (For the full list of classifiers, refer to the [Supervised learning page](https://scikit-learn.org/stable/supervised_learning.html) in the scikit-learn documentation.)

2. Using the original training data as the baseline model, fit another model with the new classifier.

3. Backtest the new model to evaluate its performance. Save a PNG image of the cumulative product of the actual returns vs. the strategy returns for this updated trading algorithm, and write your conclusions in your `README.md` file. Answer the following questions: Did this new model perform better or worse than the provided baseline model? Did this new model perform better or worse than your tuned trading algorithm?

#### Create an Evaluation Report

In the previous sections, you updated your `README.md` file with your conclusions. To accomplish this section, you need to add a summary evaluation report at the end of the `README.md` file. For this report, express your final conclusions and analysis. Support your findings by using the PNG images that you created.


In [20]:
# Imports
import pandas as pd
import numpy as np
from pathlib import Path
import hvplot.pandas
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from pandas.tseries.offsets import DateOffset
from sklearn.metrics import classification_report

---

## Establish a Baseline Performance

In this section, you’ll run the provided starter code to establish a baseline performance for the trading algorithm. To do so, complete the following steps.

Open the Jupyter notebook. Restart the kernel, run the provided cells that correspond with the first three steps, and then proceed to step four. 


### Step 1: Import the OHLCV dataset into a Pandas DataFrame.

In [21]:
# Import the OHLCV dataset into a Pandas Dataframe
ohlcv_df = pd.read_csv(
    Path("./Resources/emerging_markets_ohlcv.csv"), 
    index_col='date', 
    infer_datetime_format=True, 
    parse_dates=True
)

# Review the DataFrame
ohlcv_df.head()

,open,high,low,close,volume
date,,,,,
2015-01-21 09:30:00,23.83,23.83,23.83,23.83,100
2015-01-21 11:00:00,23.98,23.98,23.98,23.98,100
2015-01-22 15:00:00,24.42,24.42,24.42,24.42,100
2015-01-22 15:15:00,24.42,24.44,24.42,24.44,200
2015-01-22 15:30:00,24.46,24.46,24.46,24.46,200


In [22]:
# Filter the date index and close columns
signals_df = ohlcv_df.loc[:, ["close"]]

# Use the pct_change function to generate returns from close prices
signals_df["Actual Returns"] = signals_df["close"].pct_change()

# Drop all NaN values from the DataFrame
signals_df = signals_df.dropna()

# Review the DataFrame
display(signals_df.head())
display(signals_df.tail())

,close,Actual Returns
date,,
2015-01-21 11:00:00,23.98,0.006295
2015-01-22 15:00:00,24.42,0.018349
2015-01-22 15:15:00,24.44,0.000819
2015-01-22 15:30:00,24.46,0.000818
2015-01-26 12:30:00,24.33,-0.005315


,close,Actual Returns
date,,
2021-01-22 09:30:00,33.27,-0.006866
2021-01-22 11:30:00,33.35,0.002405
2021-01-22 13:45:00,33.42,0.002099
2021-01-22 14:30:00,33.47,0.001496
2021-01-22 15:45:00,33.44,-0.000896


Set baseline tuning parameters

In [23]:
'''
# DEFAULTS
months_offset = 3
short_win = 4
long_win = 100
'''

# SVM Tuned Parameters
# Set offset for train/test split
months_offset = 24

# Compute short SMA window from training split size
short_win = int(months_offset * 30 * 0.17)

# Compute long SMA window from training split size
long_win = int(months_offset * 30 * 0.35)


print(f'Training months: {months_offset}\nWindow short: {short_win}\nWindow Long: {long_win}')

Training months: 24
Window short: 122
Window Long: 251


## Step 2: Generate trading signals using short- and long-window SMA values. 

In [24]:
# Set the short window and long window
short_window = short_win
long_window = long_win

# Generate the fast and slow simple moving averages (4 and 100 days, respectively)
signals_df['SMA_Fast'] = signals_df['close'].rolling(window=short_window).mean()
signals_df['SMA_Slow'] = signals_df['close'].rolling(window=long_window).mean()

signals_df = signals_df.dropna()

# Review the DataFrame
display(signals_df.head())
display(signals_df.tail())

,close,Actual Returns,SMA_Fast,SMA_Slow
date,,,,
2015-07-14 13:00:00,23.56,0.000850,24.755738,24.665498
2015-07-14 14:00:00,23.54,-0.000849,24.736885,24.663745
2015-07-14 14:45:00,23.56,0.000850,24.718033,24.660319
2015-07-16 11:45:00,23.67,0.004669,24.699754,24.657251
2015-07-17 11:30:00,23.57,-0.004225,24.680246,24.653705


,close,Actual Returns,SMA_Fast,SMA_Slow
date,,,,
2021-01-22 09:30:00,33.27,-0.006866,30.015615,28.585777
2021-01-22 11:30:00,33.35,0.002405,30.059959,28.613386
2021-01-22 13:45:00,33.42,0.002099,30.109303,28.641633
2021-01-22 14:30:00,33.47,0.001496,30.158893,28.670239
2021-01-22 15:45:00,33.44,-0.000896,30.208074,28.698765


In [25]:
# Initialize the new Signal column
signals_df['Signal'] = 0.0

# When Actual Returns are greater than or equal to 0, generate signal to buy stock long
signals_df.loc[(signals_df['Actual Returns'] >= 0), 'Signal'] = 1

# When Actual Returns are less than 0, generate signal to sell stock short
signals_df.loc[(signals_df['Actual Returns'] < 0), 'Signal'] = -1

# Review the DataFrame
display(signals_df.head())
display(signals_df.tail())

,close,Actual Returns,SMA_Fast,SMA_Slow,Signal
date,,,,,
2015-07-14 13:00:00,23.56,0.000850,24.755738,24.665498,1.0
2015-07-14 14:00:00,23.54,-0.000849,24.736885,24.663745,-1.0
2015-07-14 14:45:00,23.56,0.000850,24.718033,24.660319,1.0
2015-07-16 11:45:00,23.67,0.004669,24.699754,24.657251,1.0
2015-07-17 11:30:00,23.57,-0.004225,24.680246,24.653705,-1.0


,close,Actual Returns,SMA_Fast,SMA_Slow,Signal
date,,,,,
2021-01-22 09:30:00,33.27,-0.006866,30.015615,28.585777,-1.0
2021-01-22 11:30:00,33.35,0.002405,30.059959,28.613386,1.0
2021-01-22 13:45:00,33.42,0.002099,30.109303,28.641633,1.0
2021-01-22 14:30:00,33.47,0.001496,30.158893,28.670239,1.0
2021-01-22 15:45:00,33.44,-0.000896,30.208074,28.698765,-1.0


In [26]:
signals_df['Signal'].value_counts()

 1.0    2280
-1.0    1792
Name: Signal, dtype: int64

In [27]:
# Calculate the strategy returns and add them to the signals_df DataFrame
signals_df['Strategy Returns'] = signals_df['Actual Returns'] * signals_df['Signal'].shift()

# Review the DataFrame
display(signals_df.head())
display(signals_df.tail())

,close,Actual Returns,SMA_Fast,SMA_Slow,Signal,Strategy Returns
date,,,,,,
2015-07-14 13:00:00,23.56,0.000850,24.755738,24.665498,1.0,NaN
2015-07-14 14:00:00,23.54,-0.000849,24.736885,24.663745,-1.0,-0.000849
2015-07-14 14:45:00,23.56,0.000850,24.718033,24.660319,1.0,-0.000850
2015-07-16 11:45:00,23.67,0.004669,24.699754,24.657251,1.0,0.004669
2015-07-17 11:30:00,23.57,-0.004225,24.680246,24.653705,-1.0,-0.004225


,close,Actual Returns,SMA_Fast,SMA_Slow,Signal,Strategy Returns
date,,,,,,
2021-01-22 09:30:00,33.27,-0.006866,30.015615,28.585777,-1.0,-0.006866
2021-01-22 11:30:00,33.35,0.002405,30.059959,28.613386,1.0,-0.002405
2021-01-22 13:45:00,33.42,0.002099,30.109303,28.641633,1.0,0.002099
2021-01-22 14:30:00,33.47,0.001496,30.158893,28.670239,1.0,0.001496
2021-01-22 15:45:00,33.44,-0.000896,30.208074,28.698765,-1.0,-0.000896


In [28]:
# Plot Strategy Returns to examine performance
(1 + signals_df['Strategy Returns']).cumprod().hvplot.line(width=1200, height=600)

:Curve   [date]   (Strategy Returns)

### Step 3: Split the data into training and testing datasets.

In [29]:
# Assign a copy of the sma_fast and sma_slow columns to a features DataFrame called X
X = signals_df[['SMA_Fast', 'SMA_Slow']].shift().dropna()

# Review the DataFrame
X.head()

,SMA_Fast,SMA_Slow
date,,
2015-07-14 14:00:00,24.755738,24.665498
2015-07-14 14:45:00,24.736885,24.663745
2015-07-16 11:45:00,24.718033,24.660319
2015-07-17 11:30:00,24.699754,24.657251
2015-07-20 15:45:00,24.680246,24.653705


In [30]:
# Create the target set selecting the Signal column and assiging it to y
y = signals_df['Signal']

# Review the value counts
y.value_counts()

 1.0    2280
-1.0    1792
Name: Signal, dtype: int64

In [31]:
# Select the start of the training period
training_begin = X.index.min()

# Display the training begin date
print(training_begin)

2015-07-14 14:00:00


In [32]:
# Select the ending period for the training data
training_end = X.index.min() + DateOffset(months=months_offset)

# Display the training end date
print(training_end)

2017-07-14 14:00:00


In [33]:
# Generate the X_train and y_train DataFrames
X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]

# Review the X_train DataFrame
X_train

,SMA_Fast,SMA_Slow
date,,
2015-07-14 14:00:00,24.755738,24.665498
2015-07-14 14:45:00,24.736885,24.663745
2015-07-16 11:45:00,24.718033,24.660319
2015-07-17 11:30:00,24.699754,24.657251
2015-07-20 15:45:00,24.680246,24.653705
...,...,...
2017-07-13 10:15:00,24.046762,23.680518
2017-07-13 11:30:00,24.054057,23.687371
2017-07-13 11:45:00,24.061066,23.694482


In [34]:
# Generate the X_test and y_test DataFrames
X_test = X.loc[training_end+DateOffset(hours=1):]
y_test = y.loc[training_end+DateOffset(hours=1):]

# Review the X_test DataFrame
X_test

,SMA_Fast,SMA_Slow
date,,
2017-07-14 15:00:00,24.085164,23.717789
2017-07-17 15:45:00,24.094426,23.726434
2017-07-18 09:30:00,24.102131,23.734243
2017-07-18 09:45:00,24.108525,23.741614
2017-07-18 10:15:00,24.115820,23.749363
...,...,...
2021-01-22 09:30:00,29.971844,28.558486
2021-01-22 11:30:00,30.015615,28.585777
2021-01-22 13:45:00,30.059959,28.613386


In [35]:
# Scale the features DataFrames
# Create a StandardScaler instance
scaler = StandardScaler()

# Apply the scaler model to fit the X-train data
X_scaler = scaler.fit(X_train)

# Transform the X_train and X_test DataFrames using the X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Step 4: Use the `SVC` classifier model from SKLearn's support vector machine (SVM) learning method to fit the training data and make predictions based on the testing data. Review the predictions.

In [36]:
# From SVM, instantiate SVC classifier model instance
svm_model = svm.SVC()
 
# Fit the model to the data using the training data
svm_model = svm_model.fit(X_train_scaled, y_train)
 
# Use the testing data to make the model predictions
svm_pred = svm_model.predict(X_test_scaled)

# Review the model's predicted values
svm_pred


array([1., 1., 1., ..., 1., 1., 1.])

### Step 5: Review the classification report associated with the `SVC` model predictions. 

In [37]:
# Use a classification report to evaluate the model using the predictions and testing data
svm_testing_report = classification_report(y_true=y_test, y_pred=svm_pred)

# Print the classification report
print(svm_testing_report)


              precision    recall  f1-score   support

        -1.0       0.47      0.35      0.40      1121
         1.0       0.57      0.69      0.62      1399

    accuracy                           0.53      2520
   macro avg       0.52      0.52      0.51      2520
weighted avg       0.52      0.53      0.52      2520



### Step 6: Create a predictions DataFrame that contains columns for “Predicted” values, “Actual Returns”, and “Strategy Returns”.

In [38]:
# Create a new empty predictions DataFrame:

# Create a predictions DataFrame
predictions_df = pd.DataFrame(index=X_test.index)

# Add the SVM model predictions to the DataFrame
predictions_df['Predicted'] = svm_pred

# Add the actual returns to the DataFrame
predictions_df['Actual Returns'] = signals_df['Actual Returns']

# Add the strategy returns to the DataFrame
predictions_df['Strategy Returns'] = signals_df['Actual Returns'] * predictions_df['Predicted']

# Review the DataFrame
predictions_df.head()

,Predicted,Actual Returns,Strategy Returns
date,,,
2017-07-14 15:00:00,1.0,0.003625,0.003625
2017-07-17 15:45:00,1.0,-0.006421,-0.006421
2017-07-18 09:30:00,1.0,-0.006058,-0.006058
2017-07-18 09:45:00,1.0,0.004876,0.004876
2017-07-18 10:15:00,1.0,0.000000,0.000000


### Step 7: Create a cumulative return plot that shows the actual returns vs. the strategy returns. Save a PNG image of this plot. This will serve as a baseline against which to compare the effects of tuning the trading algorithm.

In [39]:
# Plot the actual returns versus the strategy returns
svm_plot = (1+ predictions_df[['Actual Returns', 'Strategy Returns']]).cumprod().hvplot.line(width=800, height=400, title='SVM', xlabel='Date').opts(legend_position='top_left')
svm_plot

:NdOverlay   [Variable]
   :Curve   [date]   (value)

---

## Tune the Baseline Trading Algorithm

In this section, you’ll tune, or adjust, the model’s input features to find the parameters that result in the best trading outcomes. You’ll choose the best by comparing the cumulative products of the strategy returns.

### Step 1: Tune the training algorithm by adjusting the size of the training dataset. 

To do so, slice your data into different periods. Rerun the notebook with the updated parameters, and record the results in your `README.md` file. 

Answer the following question: What impact resulted from increasing or decreasing the training window?

### Step 2: Tune the trading algorithm by adjusting the SMA input features. 

Adjust one or both of the windows for the algorithm. Rerun the notebook with the updated parameters, and record the results in your `README.md` file. 

Answer the following question: What impact resulted from increasing or decreasing either or both of the SMA windows?

### Step 3: Choose the set of parameters that best improved the trading algorithm returns. 

Save a PNG image of the cumulative product of the actual returns vs. the strategy returns, and document your conclusion in your `README.md` file.

---

## Evaluate a New Machine Learning Classifier

In this section, you’ll use the original parameters that the starter code provided. But, you’ll apply them to the performance of a second machine learning model. 

### Step 1:  Import a new classifier, such as `AdaBoost`, `DecisionTreeClassifier`, or `LogisticRegression`. (For the full list of classifiers, refer to the [Supervised learning page](https://scikit-learn.org/stable/supervised_learning.html) in the scikit-learn documentation.)

In [40]:
# Import a new classifier from SKLearn
from sklearn.ensemble import AdaBoostClassifier

# Initiate the model instance
ada_model = AdaBoostClassifier()


### Step 2: Using the original training data as the baseline model, fit another model with the new classifier.

In [41]:
# Fit the model using the training data
ada_model = ada_model.fit(X_train_scaled, y_train)

# Use the testing dataset to generate the predictions for the new model
y_pred_ada = ada_model.predict(X_test_scaled)

# Review the model's predicted values
y_pred_ada


array([ 1.,  1.,  1., ..., -1., -1., -1.])

### Step 3: Backtest the new model to evaluate its performance. 

Save a PNG image of the cumulative product of the actual returns vs. the strategy returns for this updated trading algorithm, and write your conclusions in your `README.md` file. 

Answer the following questions: 
Did this new model perform better or worse than the provided baseline model? 
Did this new model perform better or worse than your tuned trading algorithm?

In [42]:
# Use a classification report to evaluate the model using the predictions and testing data
ada_report = classification_report(y_true=y_test, y_pred=y_pred_ada)

# Print the classification report
print(ada_report)


              precision    recall  f1-score   support

        -1.0       0.45      0.82      0.58      1121
         1.0       0.59      0.20      0.30      1399

    accuracy                           0.48      2520
   macro avg       0.52      0.51      0.44      2520
weighted avg       0.53      0.48      0.43      2520



In [43]:
# Create a new empty predictions DataFrame:

# Create a predictions DataFrame
ada_predictions_df = pd.DataFrame(index=X_test.index)

# Add the model predictions to the DataFrame
ada_predictions_df['Predicted'] = y_pred_ada

# Add the actual returns to the DataFrame
ada_predictions_df['Actual Returns'] = signals_df['Actual Returns']

# Add the strategy returns to the DataFrame
ada_predictions_df['Strategy Returns'] = ada_predictions_df['Actual Returns'] * ada_predictions_df['Predicted']

# Review the DataFrame
ada_predictions_df

,Predicted,Actual Returns,Strategy Returns
date,,,
2017-07-14 15:00:00,1.0,0.003625,0.003625
2017-07-17 15:45:00,1.0,-0.006421,-0.006421
2017-07-18 09:30:00,1.0,-0.006058,-0.006058
2017-07-18 09:45:00,1.0,0.004876,0.004876
2017-07-18 10:15:00,1.0,0.000000,0.000000
...,...,...,...
2021-01-22 09:30:00,-1.0,-0.006866,0.006866
2021-01-22 11:30:00,-1.0,0.002405,-0.002405
2021-01-22 13:45:00,-1.0,0.002099,-0.002099


In [44]:

# Plot the actual returns versus the strategy returns
ada_plot = (1+ ada_predictions_df[['Actual Returns', 'Strategy Returns']]).cumprod().hvplot.line(width=800, height=400, title='ADA', xlabel='Date').opts(legend_position='top_left')
ada_plot

:NdOverlay   [Variable]
   :Curve   [date]   (value)